In [1]:
import pandas as pd
import numpy as np
import glob
import csv

In [2]:
#Grab all csv files that were exported from fitbit 
path = r"C:\Users\jwkon\Data Learning\FitData\Fit_Project(Jun-Dec)\Raw_Data"
allFiles = glob.glob(os.path.join(path,"*.csv"))

file = []
rows = []
for f in allFiles:
    with open(f, 'r') as csvfile:
        csvfile = csvfile.read().split('\n')
        csvreader = csv.reader(csvfile)
        fields = next(csvreader)
        for row in csvreader:
            rows.append(row)
data = pd.DataFrame(rows)

In [3]:
#Filter only elements that contain data from June-December
sleep = data[data[0].str.contains('2020-06|2020-07|2020-08|2020-09|2020-10|2020-11|2020-12', na=False)]
sleep

,0,1,2,3,4,5,6,7,8,9
1,2020-08-01,"2,733","11,524",7.17,36,758,207,15,42,"1,299"
2,2020-08-02,"2,125","5,016",2.23,4,880,221,0,0,730
3,2020-08-03,"1,998","3,466",1.51,4,820,173,0,0,560
4,2020-08-04,"2,569","8,709",4.95,18,733,252,33,24,"1,272"
5,2020-08-05,"2,963","13,354",7.94,9,700,236,18,54,"1,673"
...,...,...,...,...,...,...,...,...,...,...
463,2020-09-03 10:14PM,2020-09-04 3:27AM,273,40,12,313,33,195,45,None
464,2020-09-02 10:34PM,2020-09-03 3:23AM,239,50,13,289,32,173,34,None
465,2020-09-02 2:42AM,2020-09-02 5:16AM,147,6,1,153,N/A,N/A,N/A,None
466,2020-09-01 9:49PM,2020-09-02 12:55AM,161,25,14,186,12,134,15,None


In [6]:
#Filter the dataset with the assumption that the first sleep column will display time. So, we check if it contains AM or PM
sleep = sleep[sleep[0].str.contains('AM|PM', na=False)]
sleep

,0,1,2,3,4,5,6,7,8,9
35,2020-08-30 10:35PM,2020-08-31 5:30AM,341,74,26,415,32,259,50,None
36,2020-08-29 9:31PM,2020-08-30 6:17AM,452,74,31,526,70,314,68,None
37,2020-08-29 12:43PM,2020-08-29 2:29PM,98,7,2,106,N/A,N/A,N/A,None
38,2020-08-28 11:17PM,2020-08-29 5:00AM,300,43,23,343,82,154,64,None
39,2020-08-27 9:46PM,2020-08-28 4:43AM,347,70,18,417,56,269,22,None
...,...,...,...,...,...,...,...,...,...,...
463,2020-09-03 10:14PM,2020-09-04 3:27AM,273,40,12,313,33,195,45,None
464,2020-09-02 10:34PM,2020-09-03 3:23AM,239,50,13,289,32,173,34,None
465,2020-09-02 2:42AM,2020-09-02 5:16AM,147,6,1,153,N/A,N/A,N/A,None
466,2020-09-01 9:49PM,2020-09-02 12:55AM,161,25,14,186,12,134,15,None


In [7]:
#There are more rows than the activity data and requires more filtering to be done. We will filter the second column by searching for elements that contain PM
awake_pm = sleep[sleep[1].str.contains('PM', na=False)]
awake_pm.head()

,0,1,2,3,4,5,6,7,8,9
37,2020-08-29 12:43PM,2020-08-29 2:29PM,98,7,2,106,N/A,N/A,N/A,None
44,2020-08-23 1:20PM,2020-08-23 2:36PM,70,6,1,76,N/A,N/A,N/A,None
178,2020-07-17 4:11PM,2020-07-17 5:15PM,60,3,1,63,N/A,N/A,N/A,None
191,2020-07-06 12:38PM,2020-07-06 1:45PM,60,7,0,67,N/A,N/A,N/A,None
194,2020-07-04 12:33PM,2020-07-04 1:44PM,63,8,1,71,N/A,N/A,N/A,None


In [8]:
#Combining both dataframes and dropping duplicates lets us filter out the extra 21 rows
sleep = awake_pm.append(sleep)
sleep = sleep[~sleep.index.duplicated(keep=False)]

In [9]:
sleep.head()

,0,1,2,3,4,5,6,7,8,9
35,2020-08-30 10:35PM,2020-08-31 5:30AM,341,74,26,415,32,259,50,None
36,2020-08-29 9:31PM,2020-08-30 6:17AM,452,74,31,526,70,314,68,None
38,2020-08-28 11:17PM,2020-08-29 5:00AM,300,43,23,343,82,154,64,None
39,2020-08-27 9:46PM,2020-08-28 4:43AM,347,70,18,417,56,269,22,None
40,2020-08-26 9:45PM,2020-08-27 5:19AM,382,72,29,454,56,261,65,None


In [10]:
# Sleep data is reversed, so we need to order by ascending
sleep.sort_values(by=[1], inplace=True, ascending=True)

In [11]:
# Search for Sleep column header (The fitbit export contains a csv with both activity and sleep data. Finding the 'Sleep' column helps us identify the start of the sleep dataset)
sleep_col = data[data[0].str.contains('Sleep', na=False)]
sleep_col.head(1)

,0,1,2,3,4,5,6,7,8,9
33,Sleep,None,None,None,None,None,None,None,None,None


In [12]:
#Row 33 contains the 'Sleep' column. We assume the column headers start on the next row so we assign it as such.
sleep.columns = [data.iloc[34]]

In [13]:
#Preview column headers
sleep.head(1)

34,Start Time,End Time,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,Minutes REM Sleep,Minutes Light Sleep,Minutes Deep Sleep,NaN
266,2020-06-01 10:38PM,2020-06-02 4:15AM,290,47,18,337,58,200,32,None


In [14]:
#Drop NaN and index header
sleep = sleep.iloc[:, :-1]
sleep = sleep.rename_axis(None, axis=1)
sleep = sleep.reset_index(drop=True)

In [15]:
#Preview dataframe
sleep

,Start Time,End Time,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,Minutes REM Sleep,Minutes Light Sleep,Minutes Deep Sleep
0,2020-06-01 10:38PM,2020-06-02 4:15AM,290,47,18,337,58,200,32
1,2020-06-03 1:37AM,2020-06-03 4:39AM,153,29,16,182,9,119,25
2,2020-06-03 11:05PM,2020-06-04 2:13AM,150,38,13,188,15,92,43
3,2020-06-04 11:22PM,2020-06-05 4:23AM,255,46,22,301,16,207,32
4,2020-06-05 11:13PM,2020-06-06 5:35AM,309,73,22,382,64,208,37
...,...,...,...,...,...,...,...,...,...
205,2020-12-21 10:17PM,2020-12-22 5:41AM,386,58,29,444,84,231,71
206,2020-12-22 11:06PM,2020-12-23 3:25AM,226,33,16,259,32,161,33
207,2020-12-23 9:53PM,2020-12-24 4:50AM,365,52,32,417,92,221,52
208,2020-12-24 11:48PM,2020-12-25 1:10AM,66,16,1,82,N/A,N/A,N/A


In [29]:
sleep.to_csv('Organized_Sleep_Data(Jun-Dec).csv', index=False)